In [1]:
import socket
import time


In [10]:
import socket,time,pickle
from data.Algorithem.global_repository import game_status_repository,player_status_repository,round_status_repository
class Nomal_Client():
    def __init__(self,name):
        self.name = "|"+name
        self.player_num = 2
        self.ip_list = []
        self.server_ip = "192.168.0.5"
        
    def get_list_Green_room(self):
        global game_status_repository,round_status_repository,player_status_repository
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip_address = s.getsockname()[0]
        s.close()
        myadress = ip_address
        self.ip_list = []
        for i in range(255):
            ip =  ".".join(myadress.split(".")[:-1])+f".{i}"
            # try:
            try:
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.settimeout(0.01)  # 타임아웃을 0.01초로 설정
                s.connect((ip, 12345))
                t = time.time()
                s.send(f"ping|{self.name}".encode())
                response = s.recv(1024).decode()
                print((ip,response))
                s.close()
                self.ip_list.append((ip,response))
                print(f"connecting ip : {ip}")
            except:pass
            # except (socket.timeout, socket.error):
            #     pass
        return self.ip_list
    
    def send_client(self,message, server_port=12345):
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect((self.server_ip, server_port))
        if type(message) == str:
            message +=self.name 
        t = time.time()
        try:
            self.client_socket.send(message.encode())
        except:
            self.client_socket.send(pickle.dumps(message))
        response = self.client_socket.recv(9192)
        self.client_socket.close()

        print(len(response))
        if len(response) < 3000:
            response = response.decode()
            print(f"ping :{time.time() - t}ms")
            print(f'Received from server: {response}')
            if "player_num" in response:
                self.player_num == int(response.split('|')[1])
        else:
            
            print("h")
            response = pickle.loads(response)
            print("h")
            game_status_repository = response[0]
            round_status_repository = response[1]
            player_status_repository = response[2]
            print(game_status_repository.now_turn_player, self.player_num)
            if game_status_repository.now_turn_player != self.player_num:
                self.send_client("wait")


In [11]:
client = Nomal_Client("김성훈")

In [12]:
# client.get_list_Green_room()

In [13]:
# client.send_client("exit")


In [14]:
client.send_client("JOIN")

4340
h
h
1 2


In [24]:
game_status_repository.now_turn_player = 1

In [25]:
client.send_client([game_status_repository,round_status_repository,player_status_repository])

4314
h
h
1 2
0
ping :8.381637334823608ms
Received from server: 


In [26]:
game_status_repository.now_turn_player

1

In [23]:
client.send_client([game_status_repository,round_status_repository,player_status_repository])

4314
h
h
2 2


In [15]:
game_status_repository.now_turn_player

1

In [ ]:
client.send_client([game_status_repository,round_status_repository,player_status_repository])

ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다

In [ ]:
client.send_client("exit")

Received from server: yes


In [ ]:
from data.Algorithem import *
from data.Algorithem.global_repository import game_status_repository,round_status_repository,player_status_repository

In [ ]:
import pickle

In [ ]:
g = game_status_repository = data[0]
r = round_status_repository = data[1]
p=player_status_repository = data[2]

In [ ]:
p[0].card.hand_sub_card

['Pincer', 'Basket', 'Pitchfork']

In [ ]:
game_status_repository.round_card_order

[3, 1, 0, 2, 5, 6, 4, 8, 7, 9, 10, 11, 12, 13]

In [ ]:
g.now_turn_player

0

In [ ]:
p[0].card.hand_sub_card

['JunkWarehouse', 'Manger', 'Pincer']

In [ ]:
game_status_repository.round_card_order

[2, 1, 3, 0, 6, 5, 4, 8, 7, 9, 10, 11, 12, 13]